# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
#from api_keys import g_key
from my_api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create dataframe
city_data_df = pd.read_csv('../output_data/cities_data.csv', index_col=0)

city_data_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Freeport,40.6576,-73.5832,36.00,55,90,10.36,US,1611794383
1,Jamestown,42.0970,-79.2353,21.99,93,90,4.32,US,1611794580
2,Qaanaaq,77.4840,-69.3632,-14.75,65,0,9.57,GL,1611794580
3,Albany,42.6001,-73.9662,28.99,59,99,3.00,US,1611794348
4,Safi,32.2994,-9.2372,62.69,85,0,12.55,MA,1611794581


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [30]:
# Customize the size of the figure

figure_layout = {
    'width': '600px',
    'height': '420px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Create base map
fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(10,25))

In [29]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_data_df[["Lat", "Lng"]]


# Use Humidity as the weight
humidity = city_data_df['Humidity'] 


In [31]:
# Create Humidity Heatmap layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add Heatmap layer to map
fig.add_layer(heatmap_layer)

fig



Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='1px', wi…

In [32]:
heatmap_layer.max_intensity = 100
heatmap_layer.point_radius = 3
heatmap_layer.dissipating = False

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

#### Narrow down the DataFrame to find your ideal weather condition. For example:

    * A max temperature lower than 80 degrees but higher than 70.
    * Wind speed less than 10 mph.
    * Zero cloudiness.
    * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

* Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.


In [35]:
# Create filters for desire weather condition
filter_maxtemp = (city_data_df['Max Temp']<80) & (city_data_df['Max Temp'] >75)
filter_wind = (city_data_df['Wind Speed']<10)
filter_clouds = (city_data_df['Cloudiness']==0)
filter_humid =(city_data_df['Humidity']<70) & (city_data_df['Humidity']>60)
                   

ideal_weather_pl=city_data_df[filter_maxtemp & filter_wind & filter_clouds & filter_humid]

# drop any rows with nan values
ideal_weather_pl=ideal_weather_pl.dropna()

ideal_weather_pl

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,Pedernales,18.0384,-71.7440,76.42,63,0,4.34,DO,1611794592
75,Saint-Philippe,-21.3585,55.7679,77.00,65,0,3.44,RE,1611794434
238,Vila Velha,-20.3297,-40.2925,78.80,69,0,5.75,BR,1611794601
364,Bubaque,11.2833,-15.8333,77.59,66,0,8.75,GW,1611794650
399,Saint-Pierre,-21.3393,55.4781,77.00,65,0,3.44,RE,1611794657
503,Reconquista,-29.1500,-59.6500,77.23,62,0,5.17,AR,1611794677


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
hotel_df = ideal_weather_pl[['City','Country','Lat','Lng']]

hotel_df

,City,Country,Lat,Lng
58,Pedernales,DO,18.0384,-71.7440
75,Saint-Philippe,RE,-21.3585,55.7679
238,Vila Velha,BR,-20.3297,-40.2925
364,Bubaque,GW,11.2833,-15.8333
399,Saint-Pierre,RE,-21.3393,55.4781
503,Reconquista,AR,-29.1500,-59.6500


In [37]:
# Add "Hotel Name" and "Hotel Address" columns to the DataFrame
## Note that we used "" to specify initial entry.
hotel_df["Hotel Name"]=""
hotel_df["Hotel Address"]=""

hotel_df.head()

<ipython-input-37-ce120467018b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""
<ipython-input-37-ce120467018b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Address"]=""


,City,Country,Lat,Lng,Hotel Name,Hotel Address
58,Pedernales,DO,18.0384,-71.7440,,
75,Saint-Philippe,RE,-21.3585,55.7679,,
238,Vila Velha,BR,-20.3297,-40.2925,,
364,Bubaque,GW,11.2833,-15.8333,,
399,Saint-Pierre,RE,-21.3393,55.4781,,


In [1]:
# Set parameters to search for hotels with 5000 meters.
# set up a parameters dictionary
query_params = {
    
    "radius": 5000, 
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}


for idx, row in ideal_weather_pl.iterrows():
    # get lat,lng from df
    lat=row["Lat"] 
    lng=row["Lng"]
    
    # change location each iteration while leaving original params in place
    query_params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=query_params)
    
    # convert to json
    name_address = response.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))
    
    # Use try-except to skip any missing data
    try:
        hotel_df.loc[idx, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[idx, "Hotel Address"] = name_address["results"][0]["vicinity"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

In [39]:
# Visualize to confirm hotels data appears
hotel_df

,City,Country,Lat,Lng,Hotel Name,Hotel Address
58,Pedernales,DO,18.0384,-71.7440,Vista de Águilas Ecolodge,"Calle 16 de Agosto No. 27, Pedernales"
75,Saint-Philippe,RE,-21.3585,55.7679,Les Embruns Du Baril,Réunion
238,Vila Velha,BR,-20.3297,-40.2925,Golden Tulip Porto Vitória,"Av. Nossa Sra. dos Navegantes, 635 - Enseada d..."
364,Bubaque,GW,11.2833,-15.8333,Le Mango Lodge,"Arquipélago dos Bijagós, Rua comercial"
399,Saint-Pierre,RE,-21.3393,55.4781,Le Battant Des Lames,117 Boulevard Hubert Delisle
503,Reconquista,AR,-29.1500,-59.6500,Grand Hotel,"Gral. Obligado 883, Reconquista"


In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [41]:
# Customize the size of the figure
figure_layout = {
    'width': '600px',  
    'height': '420px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(10,25))


In [42]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info, display_info_box=True)


# Add the heatmap layer to the map and add marker layer ontop of heat map
fig.add_layer(heatmap_layer)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='420px', margin='0 auto 0 auto', padding='1px', wi…